# **Hands-on Sintonia de Hiperparâmetros**

---

Sintonia de Hiperparâmetros com sklearn 

Vamos começar a olhar para alguns hiperparâmetros diversos e também para algumas tecnicas de otimização de hiperparâmetros.

Vamos seguir utilizando a base de dados **diabetes.csv** </a> que pode ser encontrada no OpenML em: https://www.openml.org/d/37. Essa base descreve o problema de predizer o resultado positivo ou negativo de um teste para diabetes aplicado em um conjunto específico de pacientes na Índia. 

Para a primeira etapa, vamos utilizar a divisão treino e teste e olhar para os dois dos três classificadores que usamos durante o hands-on de Classificação.
Note que, o processo é equivalente para as demais tarefas, tanto a de regressão quanto a de clusterização. Aqui, usaremos a classificação devido a simplicidade de interpretação do resultadoo. Mais informações sobre os classificadores podem ser encontradas na documentação do sklearn nos links abaixo. 

*   <a href = https://scikit-learn.org/stable/modules/svm.html> Support Vector Machine (SVM)</a>
*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html> Random Forest </a>


Na segunda etapa, vamos aplicar a técnica de cross validation para o processo de validação dos modelos e, em conjunto, vamos usar os métodos de grid search e random search para otimização dos hiperparâmetros. 


### *Importando* bibliotecas


In [1]:
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos

In [ ]:
#liberando acesso do colab aos arquivos no drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Carregamento dos dados

In [ ]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('/content/gdrive/My Drive/IGTI/Aulas Práticas/diabetes.csv')

In [2]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('diabetes.csv')

In [3]:
#imprimindo as 5 primeiras linhas do df para confirmação
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


*   preg: number of times pregnant
*   plas: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
*   pres: Diastolic blood pressure (mm Hg)
*   skin: Triceps skin fold thickness (mm)
*   insu: 2-Hour serum insulin (mu U/ml)
*   mass: Body mass index (weight in kg/(height in m)^2)
*   pedi: Diabetes pedigree function
*   age: Age (years)
*   class: Class variable (tested_positive or tested_negative)

In [4]:
# Verificando o numero de amostras (linhas) e features (colunas) do dataset. 
print('Amostras e Features:', df.shape)

Amostras e Features: (768, 9)


In [5]:
# Verificando quais são os tipos das features
df.columns

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age', 'class'], dtype='object')

### Pré processamento

Nesse problema, o único atributo categórico é a classe. Alguns modelos podem ter dificuldade de lidar com esse atributo. Nesse caso, precisamos transformar os valores de "tested_negative" e "tested_positivo" para 0 e 1.

Para isso, vamos fazer uma função bem simples. 

In [6]:
#criando um dicionario de dados para o mapeamento
name_to_class = {
    'tested_negative': 0,
    'tested_positive': 1
}

#substituindo os valores categóricos pelo mapeamento
df['class'] = df['class'].map(name_to_class)

#check
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Uma outra etapa importante do pré-processamento consiste na avaliação de dados faltantes. 



In [7]:
# Analisando o resumo da base
df.describe()

,preg,plas,pres,skin,insu,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Os modelos implementados no sklearn recebem como entrada para a modelagam um ou mais arrays. Dessa forma, precisamos modificar o df original para que seja possível a modelagem correta. 

Para isso, vamos separar o label das amotras, armazenar o nome das featues já que os arrays não fazem isso e depois retirar a coluna de labels do df original. Em seguida, vamos converter o df para array usando o numpy!

In [8]:
# armazenando os labels em um array
labels = np.array(df['class'])

# salvando a ordem das features
feature_list = list(df.columns)

In [9]:
# removendo a coluna de labels do df original
df = df.drop('class', axis = 1)

# check
df.columns

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')

In [10]:
# convertendo df para array
data = np.array(df)

Agora estamos quase prontos para a modelagem em si!

Vamos usar a função <a href = http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html> train_test_split </a> e olhar para para algumas combinações de parâmetros dos modelos que selecionamos. 

In [11]:
# importar train_test_split do scikitlearn 
from sklearn.model_selection import train_test_split

# aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)

# Support Vector Machine SVM

Vamos dar uma olhada agora no comportamento do SVM

In [12]:
#importar o modelo SVM
from sklearn.svm import SVC

In [13]:
# treinando o SVM com kernel RBF
classifier1 = SVC(kernel='rbf').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions1_labels = classifier1.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions1_labels})  
#p.head(10)

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [14]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions1_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions1_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions1_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions1_labels))

Matriz de Confusão
 [[123   0]
 [ 69   0]]

Acurácia
 0.640625

F1
 0.0

AUCROC
 0.5


D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Vamos tentar melhorar o SVM? 

Vamos trocar o kernel gaussiano pelo sigmoidal

In [15]:
# treinando o SVM com kernel sigmoidal
classifier2 = SVC(kernel='sigmoid').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions2_labels = classifier2.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions2_labels})  
#p.head(10)

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [16]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions2_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions2_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions2_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions2_labels))

Matriz de Confusão
 [[123   0]
 [ 69   0]]

Acurácia
 0.640625

F1
 0.0

AUCROC
 0.5


D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Vamos tentar outra opção!


In [17]:
# treinando o SVM com kernel rbf, com largura da gaussiana alterada
classifier3 = SVC(kernel='rbf', gamma = 'auto').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions3_labels = classifier3.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions3_labels})  
#p.head(10)

In [18]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions3_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions3_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions3_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions3_labels))

Matriz de Confusão
 [[123   0]
 [ 69   0]]

Acurácia
 0.640625

F1
 0.0

AUCROC
 0.5


D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Ultimo teste!

In [19]:
# treinando o SVM com kernel rbf, com largura da gaussiana alterada
classifier4 = SVC(C = 0.1, kernel='rbf', gamma = 0.0001).fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions4_labels = classifier4.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions4_labels})  
#p.head(10)

In [20]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions4_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions4_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions4_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions4_labels))

Matriz de Confusão
 [[112  11]
 [ 43  26]]

Acurácia
 0.71875

F1
 0.49056603773584906

AUCROC
 0.6436903499469776


# Random Forest Classifiers


Vamos agora modelar o nosso problema utilizando o Random Forest Classifier e testar algumas combinações de hiperparâmetros!

In [21]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# treinando o modelo 
classifier5 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions5_labels = classifier5.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [22]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions5_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions5_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions5_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions5_labels))

Matriz de Confusão
 [[97 26]
 [24 45]]

Acurácia
 0.7395833333333334

F1
 0.6428571428571428

AUCROC
 0.72039589961117


Eita...

Será que o nosso resultado pode ser melhor?

Vamos trocar random_state e dar uma olhada em como a característica aleatoria do RF influencia no resultado.


In [23]:
# treinando o modelo 
classifier6 = RandomForestClassifier(n_estimators= 10, random_state = 124).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions6_labels = classifier6.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [24]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions6_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions6_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions6_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions6_labels))

Matriz de Confusão
 [[100  23]
 [ 24  45]]

Acurácia
 0.7552083333333334

F1
 0.656934306569343

AUCROC
 0.7325910215623894


Vamos tentar aumentar o número de estimadores

In [25]:
# treinando o modelo 
classifier7 = RandomForestClassifier(n_estimators= 100, random_state = 42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions7_labels = classifier7.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [26]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions7_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions7_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions7_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions7_labels))

Matriz de Confusão
 [[96 27]
 [24 45]]

Acurácia
 0.734375

F1
 0.6382978723404256

AUCROC
 0.7163308589607635


In [27]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# treinando o modelo 
classifier8 = RandomForestClassifier(n_estimators= 100, bootstrap = False, random_state = 42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions8_labels = classifier8.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions8_labels})  
#p.head(10)

In [28]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions8_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions8_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions8_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions8_labels))

Matriz de Confusão
 [[93 30]
 [24 45]]

Acurácia
 0.71875

F1
 0.6250000000000001

AUCROC
 0.704135737009544


#### Qual seria o custo, isto é, qual o tempo que nós perdiaríamos ao ter que calibrar manualmente, de forma aleatória ou sistemática, todos os modelos que criamos? 

Só não!

Vamos dar uma olhada em como funciona o Random Search e o Grid Search aplicados? SIIIIIIIIIIIM

Vamos aproveitar para usar a validação cruzada estratificada!

O sklearn tem funções de Grid Search e Random Search implementadas que já fazem o processo de validação cruzada do modelo. 
Mais informações sobre esses métodos podem ser encontradas nos links abaixo!

*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV> Grid Search CV </a>
*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV> Random Search CV</a>

Vamos testa-las?

## Grid Search



**Vamos testar o Grid Search com kfold estratificado e o SVM**

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [32]:
#definindo hiperparâmetros
hiperparam = {'kernel':('sigmoid', 'rbf'), 'C':[0.01, 1, 10]}

#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#instânciando meu classificador
classifier = SVC()

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
grid_cv = GridSearchCV(classifier, hiperparam, cv = cv_strat, scoring = f1)
grid_cv.fit(data, labels)

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to n

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'kernel': ('sigmoid', 'rbf'), 'C': [0.01, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score), verbose=0)

In [33]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', grid_cv.cv_results_)


Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.00930278, 0.04836261, 0.01035354, 0.04697073, 0.00983362,
       0.04974012]), 'std_fit_time': array([0.00247808, 0.00719847, 0.00288254, 0.00822491, 0.00302   ,
       0.00928086]), 'mean_score_time': array([0.00212173, 0.00562322, 0.00185125, 0.00490558, 0.00193851,
       0.00487204]), 'std_score_time': array([0.00054118, 0.00158089, 0.00054233, 0.00071017, 0.00075365,
       0.00092939]), 'param_C': masked_array(data=[0.01, 0.01, 1, 1, 10, 10],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['sigmoid', 'rbf', 'sigmoid', 'rbf', 'sigmoid', 'rbf'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 0.01, 'kernel': 'sigmoid'}, {'C': 0.01, 'kernel': 'rbf'}, {'C': 1, 'kernel': 'sigmoid'}, {'C': 1, 'kernel': 'rbf'}, {'C': 10, 'kernel': 'sigmoid

In [34]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', grid_cv.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', grid_cv.best_params_)

print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', grid_cv.best_estimator_)

Melhor resultado f1: 0.0


Melhor configuração de hiperparâmetros: {'C': 0.01, 'kernel': 'sigmoid'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='sigmoid', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


**Vamos agora repetir o processo para o Random Forest**


In [35]:
#definindo hiperparâmetros
#hiperparam1 = {'n_estimators':[10, 100, 1000]}
hiperparam1 = {'n_estimators':[10, 100, 1000], 'bootstrap': (True, False)}
#hiperparam1 = {'n_estimators':[10, 100, 1000], 'bootstrap': (True, False), 'criterion': ('gini', 'entropy')}

#instânciando meu classificador
classifier1 = RandomForestClassifier()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
grid_cv1 = GridSearchCV(classifier1, hiperparam1, cv = cv_strat, scoring = f1)
grid_cv1.fit(data, labels)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 100, 1000], 'bootstrap': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score), verbose=0)

In [36]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', grid_cv1.cv_results_)


Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.01329043, 0.12503524, 1.20799363, 0.01704376, 0.16836636,
       1.4965229 ]), 'std_fit_time': array([0.00168121, 0.01162909, 0.02035893, 0.0011747 , 0.02223689,
       0.06461461]), 'mean_score_time': array([0.00172381, 0.00779912, 0.07153549, 0.00137765, 0.00900207,
       0.07125366]), 'std_score_time': array([0.00067253, 0.00112358, 0.0052732 , 0.00048776, 0.00383594,
       0.00969934]), 'param_bootstrap': masked_array(data=[True, True, True, False, False, False],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'bootstrap': True, 'n_estimators': 10}, {'bootstrap': True, 'n_estimators': 100}, {'bootstrap': True, 'n_estimators': 1000}, {'bootstrap': False, 

In [37]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', grid_cv1.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', grid_cv1.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', grid_cv1.best_estimator_)

Melhor resultado f1: 0.6387891111076285


Melhor configuração de hiperparâmetros: {'bootstrap': False, 'n_estimators': 1000}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


## Random Search

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


In [44]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions = dict(kernel = ['sigmoid', 'rbf'], 
                     C = uniform(loc=0, scale=10))

#instânciando meu classificador
classifier = SVC()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv = RandomizedSearchCV(classifier, distributions, cv = cv_strat, scoring = f1, random_state = 42, n_iter = 10)
random_cv.fit(data, labels)

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'sc

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Users\elain\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\ela

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
          error_score='raise-deprecating',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'kernel': ['sigmoid', 'rbf'], 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C272843D30>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=make_scorer(f1_score),
          verbose=0)

In [45]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', random_cv.cv_results_)

Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.00650303, 0.03951728, 0.00630486, 0.00730114, 0.03435154,
       0.03915203, 0.03892083, 0.03438857, 0.03855271, 0.03231223]), 'std_fit_time': array([0.00067812, 0.00242362, 0.00063183, 0.00220112, 0.00180027,
       0.00291224, 0.00235307, 0.00143944, 0.00151541, 0.00288432]), 'mean_score_time': array([0.00140417, 0.00436108, 0.0013247 , 0.00150361, 0.00408819,
       0.00399196, 0.00447016, 0.00442824, 0.00381341, 0.00404897]), 'std_score_time': array([0.00045731, 0.00089812, 0.00045119, 0.00047098, 0.00069207,
       0.00073545, 0.00117404, 0.00101693, 0.00060278, 0.00057615]), 'param_C': masked_array(data=[3.745401188473625, 1.834347898661638,
                   5.986584841970366, 4.458327528535912,
                   0.5808361216819946, 3.337086111390218,
                   7.080725777960454, 0.5641157902710026,
                   8.324426408004218, 0.007787658410143283],
             mask=[False, False, False

In [46]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv.best_estimator_)

Melhor resultado f1: 0.0


Melhor configuração de hiperparâmetros: {'C': 3.745401188473625, 'kernel': 'sigmoid'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=3.745401188473625, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='sigmoid', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


Vamos testar agora o mesmo processo para o RF

In [47]:
from scipy.stats import randint

In [48]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions1 = dict(n_estimators = randint(10, 100),
                      bootstrap = [True, False],
                      criterion = ['gini', 'entropy'])

#instânciando meu classificador
classifier1 = RandomForestClassifier(random_state = 42)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv1 = RandomizedSearchCV(classifier1, distributions1, cv = cv_strat, scoring = f1, random_state = 42, n_iter = 10)
random_cv1.fit(data, labels)

D:\Users\elain\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
          error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C272843860>, 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=make_scorer(f1_score),
          verbose=0)

In [49]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', random_cv1.cv_results_)

Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.0391547 , 0.12130558, 0.12025564, 0.12046058, 0.06205404,
       0.09699383, 0.07930241, 0.14197602, 0.07267945, 0.06113634]), 'std_fit_time': array([0.00631872, 0.03687798, 0.03107159, 0.02856065, 0.01669747,
       0.00797051, 0.0068281 , 0.01382848, 0.00945426, 0.00657684]), 'mean_score_time': array([0.00262654, 0.00747368, 0.00776639, 0.00854604, 0.00395958,
       0.00578096, 0.00412185, 0.00595074, 0.00435288, 0.0036442 ]), 'std_score_time': array([0.00076746, 0.00485098, 0.00188432, 0.0054269 , 0.00056651,
       0.00103827, 0.00144802, 0.00063012, 0.00087585, 0.00090832]), 'param_bootstrap': masked_array(data=[True, True, True, True, False, True, False, False,
                   False, False],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_criterion': masked_array(data=['entropy', 'entropy'

In [50]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv1.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv1.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv1.best_estimator_)

Melhor resultado f1: 0.6337798895837207


Melhor configuração de hiperparâmetros: {'bootstrap': False, 'criterion': 'gini', 'n_estimators': 42}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=42, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


O Processo de parametrização de um modelo nem sempre é simples. 

Existem alguns métodos que podem nos ajudar a suavizar o custo da otimização.

Entretanto, é preciso saber quais os hiperperâmetros mais interessantes para calibrar e qual a faixa de valores que faz mais sentido. 

Nesse hands-on falamos de Grid e Random Search. A escolha do método fica por sua conta. 
É preciso pensar com cuidado e avaliar as soluções encontradas pelos métodos. 

# Chegamos ao final do módulo! 

# Vejo vocês de novo em breve!